# Lab Three: Extending Logistic Regression
 

#### Everett Cienkus, Blake Miller, Colin Weil

### 1. Preparation and Overview

#### 1.1 Business Case

Explain the task and what business-case or use-case it is designed to solve (or designed to investigate). Detail exactly what the classification task is and what parties would be interested in the results. For example, would the model be deployed or used mostly for offline analysis? 

#### 1.2 Preparation of Data

In [1]:
import pandas as pd
import numpy as np

# Define and prepare your class variables.
df = pd.read_csv('star_dataset/star_classification.csv')
df = df.dropna()

X = df.drop(columns = ['obj_ID','run_ID','rerun_ID','field_ID','spec_obj_ID', 'MJD', 'class','plate', 'fiber_ID','cam_col' ])
#X = df.drop(columns = ['obj_ID','run_ID','rerun_ID','field_ID','spec_obj_ID', 'MJD', 'class'])
y = df['class']

# Use proper variable representations (int, float, one-hot, etc.).
# Use pre-processing methods (as needed) for dimensionality reduction, 
# scaling, etc. Remove variables that are not needed/useful for the analysis. 
# Describe the final dataset that is used for classification/regression
display(X.info())
display(y.info())
# (include a description of any newly formed variables you created).
# MAKE SURE TO NORMALIZE VALUES

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   alpha     100000 non-null  float64
 1   delta     100000 non-null  float64
 2   u         100000 non-null  float64
 3   g         100000 non-null  float64
 4   r         100000 non-null  float64
 5   i         100000 non-null  float64
 6   z         100000 non-null  float64
 7   redshift  100000 non-null  float64
dtypes: float64(8)
memory usage: 6.1 MB


None

<class 'pandas.core.series.Series'>
RangeIndex: 100000 entries, 0 to 99999
Series name: class
Non-Null Count   Dtype 
--------------   ----- 
100000 non-null  object
dtypes: object(1)
memory usage: 781.4+ KB


None

#### 1.3 Division of Trainig and Testing Data

In [2]:
# Divide your data into training and testing data using an 80% training 
# and 20% testing split. Use the cross validation modules that are part 
# of scikit-learn.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8)

unique_ytrain, counts_ytrain = np.unique(y_train, return_counts=True)
print(np.asarray((unique_ytrain, counts_ytrain)).T)

[['GALAXY' 47480]
 ['QSO' 15354]
 ['STAR' 17166]]


Argue "for" or "against" splitting your data using an 80/20 split. That is, why is the 80/20 split appropriate (or not) for your dataset?  

In [3]:
### 2. Modeling

#### 2.1 One-Versus-All Logistic Regression Classifier

In [4]:
from scipy.special import expit
from sklearn.metrics import accuracy_score

class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C
        # internally we will store the weights as self.w_ to keep with sklearn conventions

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'

    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term

    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))

    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through

        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C

        return gradient

    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1

    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction


    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros

        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate
            # add bacause maximizing

# for this, we won't perform our own BFGS implementation
# (it takes a fair amount of code and understanding, which we haven't setup yet)
# luckily for us, scipy has its own BFGS implementation:
from scipy.optimize import fmin_bfgs # maybe the most common bfgs algorithm in the world
from numpy import ma
class BFGSBinaryLogisticRegression(BinaryLogisticRegression):

    @staticmethod
    def objective_function(w,X,y,C):
        g = expit(X @ w)
        # invert this because scipy minimizes, but we derived all formulas for maximzing
        return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(w**2)
        #-np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w,X,y,C):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient

    # just overwrite fit function
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                            np.zeros((num_features,1)), # starting point
                            fprime=self.objective_gradient, # gradient function
                            args=(Xb,y,self.C), # extra args for gradient and objective function
                            gtol=1e-03, # stopping criteria for gradient, |v_k|
                            maxiter=self.iters, # stopping criteria iterations
                            disp=False)

        self.w_ = self.w_.reshape((num_features,1))

class StochasticLogisticRegression(BinaryLogisticRegression):
    # stochastic gradient calculation
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through

        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C

        return gradient

class MultiClassLogisticRegression:
    def __init__(self, eta, iterations=20,
                 C=0.0001,
                 solver=BFGSBinaryLogisticRegression):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.solver = solver
        self.classifiers_ = []
        # internally we will store the weights as self.w_ to keep with sklearn conventions

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'

    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = np.array(y==yval).astype(int) # create a binary problem

            # train the binary classifier for this class

            hblr = self.solver(eta=self.eta,iterations=self.iters,C=self.C)
            hblr.fit(X,y_binary)

            # add the trained classifier to the list
            self.classifiers_.append(hblr)

        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

    def predict_proba(self,X):
        probs = []
        for hblr in self.classifiers_:
            probs.append(hblr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier

        return np.hstack(probs) # make into single matrix

    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row

In [5]:
%%time
lr = MultiClassLogisticRegression(eta=0.05,
                                  iterations=100,
                                  C=0.0001,
                                  solver=BFGSBinaryLogisticRegression
                                  )
#np.hstack((np.ones((X.shape[0],1)),X))
lr.fit(X_train,y_train)
print(lr)
yhat = lr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))
unique_yhat, counts_yhat = np.unique(yhat, return_counts=True)
unique_y, counts_y = np.unique(y_test, return_counts=True)
print(np.asarray((unique_yhat, counts_yhat)).T)
print(np.asarray((unique_y, counts_y)).T)

MultiClass Logistic Regression Object with coefficients:
[[ 3.61687470e-01 -1.87895887e-05 -4.98046442e-03 -3.91212130e-02
   1.78377903e+00 -7.21251718e-01 -9.06240712e-01 -2.08431902e-01
  -3.84491714e-01]
 [-2.25738320e-01  1.73158081e-03  1.42295706e-02 -6.63518840e-01
  -1.16761074e-01 -4.56072270e-01  2.44789877e-01  8.05183383e-01
   5.25357508e+00]
 [-1.44139583e+00 -1.80212078e-04 -3.17430076e-03  2.67415150e-03
  -5.16699629e-01  1.40978064e+00 -6.80507140e-01 -6.54310907e-02
  -2.05261988e+01]]
Accuracy of:  0.8912
[['GALAXY' 12499]
 ['QSO' 3559]
 ['STAR' 3942]]
[['GALAXY' 11965]
 ['QSO' 3607]
 ['STAR' 4428]]
CPU times: total: 7.78 s
Wall time: 4.75 s


In [6]:
%%time
from sklearn.linear_model import LogisticRegression as SKLogisticRegression

lr_sk = SKLogisticRegression(solver='liblinear') # all params default

lr_sk.fit(X,y)
print(np.hstack((lr_sk.intercept_[:,np.newaxis],lr_sk.coef_)))
yhat = lr_sk.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

[[ 4.59395586e-01 -6.06488062e-05 -4.83932681e-03 -5.42363750e-02
   1.82731013e+00 -7.45739264e-01 -9.61005199e-01 -1.62208839e-01
  -3.74037808e-01]
 [-1.84778525e-01  1.08558961e-03  1.05928725e-02 -7.23158024e-01
  -1.73494110e-01 -4.05275465e-01  2.85626329e-01  8.98827708e-01
   3.84726209e+00]
 [-4.27124241e+00 -2.26820578e-04  7.81354749e-05  3.66087194e-01
  -4.10280526e-01  1.53665567e-01  1.72150229e-01  4.38593362e-02
  -5.28461570e+01]]
Accuracy of:  0.93088
CPU times: total: 11.5 s
Wall time: 12.1 s


C:\Users\tallb\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### 2. Training Classifier for Good Generalization Performance

Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?

#### 2.3 Comparing Best Performing Procedure to Scikit-Learn

In [7]:
# Visualize the performance differences in terms of training time and classification performance.

Discuss the results. 

### 3. Deployment

Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

### 4. BFGS (Can change but thought this would be better)

In [8]:
# Implementation of BFGS

Compare your performance accuracy and runtime to the BFGS implementation in SciPy (that we used in lecture). 
